In [29]:
import requests
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Configuración de la API
API_KEY = "9099e7bdbb9c6721aeb6c5f8f35f47b2dbd992748393b8ae8d9064fd0591818e"
URL = "https://min-api.cryptocompare.com/data/v2/histoday"
LIMIT = 2000  # Máximo número de registros por solicitud

def fetch_crypto_data(symbol, currency, limit, api_key):
    all_data = []
    to_ts = None
    count = 0  # Contador para verificar el progreso

    while True:
        params = {
            "fsym": symbol,
            "tsym": currency,
            "limit": limit - 1,
            "toTs": to_ts,
            "api_key": api_key
        }
        response = requests.get(URL, params=params)
        response.raise_for_status()  # Validar que no haya errores
        data = response.json()["Data"]["Data"]

        if not data:
            print("No hay más datos disponibles.")
            break
		
        len_data = len(data)
        all_data.extend(data)
        to_ts = data[0]["time"]  # Actualizar para el siguiente bloque


        count += 1
        print(f"Iteración {count}: Datos descargados hasta {pd.to_datetime(to_ts, unit='s')} - {len_data} registros")
        
        if pd.to_datetime(to_ts, unit='s').strftime("%Y-%m-%d") < '2010-09-23':
            break
        # Detener cuando ya no hay más datos
        if len(data) < limit:
            break

    return pd.DataFrame(all_data)

# Descargar datos históricos de Bitcoin
df = fetch_crypto_data("BTC", "USD", LIMIT, API_KEY)


# Convertir el timestamp a fecha y renombrar columnas
df["time"] = pd.to_datetime(df["time"], unit="s")
df.rename(columns={"close": "Close"}, inplace=True)

df = df[df['time']>='2010-09-23']
df = df.sort_values(by='time', ascending=True)

# Guardar el DataFrame en un archivo CSV para futuras ejecuciones rápidas
df.to_csv("btc_historical_data.csv", index=False)
print("Datos descargados y guardados en 'btc_historical_data.csv'.")


Iteración 1: Datos descargados hasta 2019-06-20 00:00:00 - 2000 registros
Iteración 2: Datos descargados hasta 2013-12-29 00:00:00 - 2000 registros
Iteración 3: Datos descargados hasta 2008-07-09 00:00:00 - 2000 registros
Datos descargados y guardados en 'btc_historical_data.csv'.


In [ ]:
# Generar las bandas del Rainbow Chart
def rainbow_bands(data_dates, price_range):
    bands = []
    multipliers = [0.02, 0.05, 0.1, 0.2, 0.5, 1, 2, 5, 10, 20]  # Ajusta según tus preferencias
    colors = ['blue', 'cyan', 'green', 'lime', 'yellow', 'orange', 'red', 'darkred', 'purple', 'magenta']
    for multiplier, color in zip(multipliers, colors):
        bands.append(dict(
            x=data_dates,
            y=price_range * multiplier,
            color=color
        ))
    return bands


def var_log(x,base):
    return np.log(x) / np.log(base)

# Crear rango de precios para las bandas
price_start = df["Close"].min()
price_end = df["Close"].max()
price_range_log = np.logspace(np.log10(price_start), np.log10(price_end), len(df), base=5.0)
price_range = np.linspace(df["Close"].min(), df["Close"].max(), len(df)) 
 


bands = rainbow_bands(df["time"], price_range)
df['price_range'] = price_range
df['price_range_log'] = price_range_log
# Crear el gráfico con Plotly
fig = go.Figure()

# Añadir las bandas al gráfico
for band in bands:
    fig.add_trace(go.Scatter(
        x=band["x"],
        y=band["y"],
        mode="lines",
        line=dict(color=band["color"], width=0),
        fill="tonexty",
        name=f"Zona {band['color']}",
        opacity=0.5
    ))

# Añadir los datos históricos de Bitcoin
fig.add_trace(go.Scatter(
    x=df["time"],
    y=df["Close"],
    mode="lines",
    line=dict(color="black", width=2),
    name="BTC-USD"
))

# Añadir los datos históricos de Bitcoin
fig.add_trace(go.Scatter(
    x=df["time"],
    y=df["price_range_log"],
    mode="lines",
    line=dict(color="blue", width=2),
    name="Price Range"
))

# Configurar ejes y diseño
fig.update_layout(
    title="Bitcoin Rainbow Chart (Historia Completa)",
    xaxis=dict(title="Fecha", type="date", range=[df["time"].min(), df["time"].max()]),
    yaxis=dict(title="Precio (USD)", type="log"),
    legend=dict(title="Zonas"),
    template="plotly_white",
    hovermode="x unified"
)

# Mostrar el gráfico
fig.show()

df

,time,high,low,open,volumefrom,volumeto,Close,conversionType,conversionSymbol,days_since_start,price_range,price_range_log
4806,2010-09-23,0.06300,0.06150,0.06220,15505.33,9.700900e+02,0.06231,direct,,1,0.061110,0.061110
4807,2010-09-24,0.06240,0.06209,0.06231,684.38,4.266000e+01,0.06220,direct,,2,19.546908,0.061247
4808,2010-09-25,0.06240,0.06170,0.06220,2152.50,1.330500e+02,0.06202,direct,,3,39.032706,0.061385
4809,2010-09-26,0.06228,0.06186,0.06202,12058.13,7.482000e+02,0.06220,direct,,4,58.518504,0.061523
4810,2010-09-27,0.06228,0.06190,0.06220,10752.25,6.685300e+02,0.06221,direct,,5,78.004302,0.061661
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2024-12-05,104028.51000,91741.97000,98751.87000,119875.78,1.204144e+10,97053.82000,direct,,5188,101111.866808,6968.814323
1996,2024-12-06,102088.57000,96427.00000,97053.82000,56899.66,5.653979e+09,99897.97000,direct,,5189,101131.352606,6984.470122
1997,2024-12-07,100578.80000,99025.63000,99897.97000,17306.94,1.728103e+09,99926.38000,direct,,5190,101150.838404,7000.161093
1998,2024-12-08,101430.60000,98730.22000,99926.38000,17851.82,1.785051e+09,101189.81000,direct,,5191,101170.324202,7015.887315
